In [32]:
import requests
import pandas as pd
import io
import json
import configparser

In [45]:
config = configparser.RawConfigParser()
config.read('credentials.conf')

['credentials.conf']

# 1 - API DE STREAMING

In [34]:
url = "https://streaming-availability.p.rapidapi.com/search/basic"

querystring = {"country":"us","service":"netflix","type":"movie","genre":"18","page":"1","output_language":"en","language":"en"}

In [35]:
headers = {
	"X-RapidAPI-Host": "streaming-availability.p.rapidapi.com",
	"X-RapidAPI-Key": config["RAPIDAPI"]["KEY"]
}

r = requests.request("GET", url, headers=headers, params=querystring)


In [36]:
pretty_json = json.loads(r.text)
print (json.dumps(pretty_json, indent=2))

{
  "results": [
    {
      "imdbID": "tt9850370",
      "tmdbID": "610643",
      "imdbRating": 65,
      "imdbVoteCount": 1511,
      "tmdbRating": 69,
      "backdropPath": "/pYziM5SEmptPW0LdNhWvjzR2zD1.jpg",
      "backdropURLs": {
        "1280": "https://image.tmdb.org/t/p/w1280/pYziM5SEmptPW0LdNhWvjzR2zD1.jpg",
        "300": "https://image.tmdb.org/t/p/w300/pYziM5SEmptPW0LdNhWvjzR2zD1.jpg",
        "780": "https://image.tmdb.org/t/p/w780/pYziM5SEmptPW0LdNhWvjzR2zD1.jpg",
        "original": "https://image.tmdb.org/t/p/original/pYziM5SEmptPW0LdNhWvjzR2zD1.jpg"
      },
      "originalTitle": "#AnneFrank. Parallel Stories",
      "genres": [
        18,
        36
      ],
      "countries": [
        "IT"
      ],
      "year": 2019,
      "runtime": 92,
      "cast": [
        "Helen Mirren",
        "Anne Frank",
        "Martina Gatti",
        "Arianna Szorenyi"
      ],
      "significants": [
        "Sabina Fedeli",
        "Anna Migotto"
      ],
      "title": "#AnneFr

In [37]:
pd.DataFrame.from_dict(pretty_json, orient='columns')

,results,total_pages
0,"{'imdbID': 'tt9850370', 'tmdbID': '610643', 'i...",60
1,"{'imdbID': 'tt5616176', 'tmdbID': '710356', 'i...",60
2,"{'imdbID': 'tt0478087', 'tmdbID': '8065', 'imd...",60
3,"{'imdbID': 'tt7280898', 'tmdbID': '474354', 'i...",60
4,"{'imdbID': 'tt2933544', 'tmdbID': '286521', 'i...",60
5,"{'imdbID': 'tt6142496', 'tmdbID': '500634', 'i...",60
6,"{'imdbID': 'tt3799372', 'tmdbID': '323674', 'i...",60
7,"{'imdbID': 'tt13055780', 'tmdbID': '768520', '...",60


## 2 - API DO TWITTER

In [38]:
!pip install tweepy

You should consider upgrading via the 'C:\Users\LuizErnestoFerreiraA\anaconda3\python.exe -m pip install --upgrade pip' command.


In [47]:
import tweepy

client = tweepy.Client(config["TWITTER"]["TOKEN"])
query = "Miranha"


start = '2022-04-17T00:00:00Z'
end = '2022-04-21T23:59:00Z'

# https://docs.tweepy.org/en/stable/client.html
# https://docs.tweepy.org/en/stable/client.html#tweepy.Client.get_recent_tweets_count

response = client.get_recent_tweets_count(query, granularity="hour", start_time = start, end_time= end)

for count in response.data:
    print(count)

{'end': '2022-04-17T01:00:00.000Z', 'start': '2022-04-17T00:00:00.000Z', 'tweet_count': 27}
{'end': '2022-04-17T02:00:00.000Z', 'start': '2022-04-17T01:00:00.000Z', 'tweet_count': 38}
{'end': '2022-04-17T03:00:00.000Z', 'start': '2022-04-17T02:00:00.000Z', 'tweet_count': 23}
{'end': '2022-04-17T04:00:00.000Z', 'start': '2022-04-17T03:00:00.000Z', 'tweet_count': 30}
{'end': '2022-04-17T05:00:00.000Z', 'start': '2022-04-17T04:00:00.000Z', 'tweet_count': 9}
{'end': '2022-04-17T06:00:00.000Z', 'start': '2022-04-17T05:00:00.000Z', 'tweet_count': 10}
{'end': '2022-04-17T07:00:00.000Z', 'start': '2022-04-17T06:00:00.000Z', 'tweet_count': 16}
{'end': '2022-04-17T08:00:00.000Z', 'start': '2022-04-17T07:00:00.000Z', 'tweet_count': 2}
{'end': '2022-04-17T09:00:00.000Z', 'start': '2022-04-17T08:00:00.000Z', 'tweet_count': 1}
{'end': '2022-04-17T10:00:00.000Z', 'start': '2022-04-17T09:00:00.000Z', 'tweet_count': 6}
{'end': '2022-04-17T11:00:00.000Z', 'start': '2022-04-17T10:00:00.000Z', 'tweet_coun

In [48]:
df = pd.DataFrame.from_dict(response.data, orient='columns')
df['end'] = pd.to_datetime(df.end)
df['start'] = pd.to_datetime(df.start)
df.head()

,end,start,tweet_count
0,2022-04-17 01:00:00+00:00,2022-04-17 00:00:00+00:00,27
1,2022-04-17 02:00:00+00:00,2022-04-17 01:00:00+00:00,38
2,2022-04-17 03:00:00+00:00,2022-04-17 02:00:00+00:00,23
3,2022-04-17 04:00:00+00:00,2022-04-17 03:00:00+00:00,30
4,2022-04-17 05:00:00+00:00,2022-04-17 04:00:00+00:00,9


In [49]:
df['hour'] = df.end.dt.hour
df['day'] = df.end.dt.day
df = df[["day","hour","tweet_count"]]
df

,day,hour,tweet_count
0,17,1,27
1,17,2,38
2,17,3,23
3,17,4,30
4,17,5,9
...,...,...,...
115,21,20,23
116,21,21,30
117,21,22,39
118,21,23,27


In [50]:
response = client.search_recent_tweets(query)
print(response.meta)

tweets = response.data
for tweet in tweets:
    print(tweet.id)
    print(tweet.text)

{'newest_id': '1517604023960014857', 'oldest_id': '1517599446900563969', 'result_count': 10, 'next_token': 'b26v89c19zqg8o3fpytnyw9psop29f46h6tmgdx128g3h'}
1517604023960014857
RT @tralhasdojon: Ten algum editor na Marvel cujo trabalho é incentivar o máximo de novas roupas aleatórias pro Miranha (que ele vai usar d…
1517603727267569665
Lancei o miranha
1517603092031754241
@Gabrirul @liciawebber Perdi a paciência com a Marvel faz tempo maklgmklaeg acho que a única coisa da Marvel que eu assisti desde Endgame foi o filme do Miranha, e só
1517602838783967233
n queria falar isso mas batman arkham knight é melhor que os jogos do miranha ate agr
1517602838452613120
@RTFKT_MLNTH @bclever_nft @midnight235959 @TGIMCREW @NFTYbird @rx_miranha @MNLTH is opening!!!! @GGtesting1978 @dmoneybagsLLC @Fajni3 @Raerol420 @CryptoCorson_ @mypvck @BikalpaUpadhya1 @CryptoAstronau8 @hardriversz
1517602543966240769
tô vendo miranha pela 5 vez
1517601283426955265
RT @tralhasdojon: Ten algum editor na Marvel cujo 

In [9]:
#!pip install pyarrow

You should consider upgrading via the 'C:\Users\LuizErnestoFerreiraA\anaconda3\python.exe -m pip install --upgrade pip' command.


In [11]:
import pandas as pd
pd.read_parquet("..\data\imdb_top_1000.parquet", engine="pyarrow")

,title,released_year,certificate,runtime,genre1,genre2,genre3,imdb,overview,meta_score,director,star1,star2,star3,star4,votes,gross
0,The Shawshank Redemption,1994,A,142,Drama,None,None,9.3,Two imprisoned men bond over a number of years...,80,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,28341469
1,The Godfather,1972,A,175,Crime,Drama,None,9.2,An organized crime dynasty's aging patriarch t...,100,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,134966411
2,The Dark Knight,2008,UA,152,Action,Crime,Drama,9.0,When the menace known as the Joker wreaks havo...,84,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,534858444
3,The Godfather: Part II,1974,A,202,Crime,Drama,None,9.0,The early life and career of Vito Corleone in ...,90,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,57300000
4,12 Angry Men,1957,U,96,Crime,Drama,None,9.0,A jury holdout attempts to prevent a miscarria...,96,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,4360000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Breakfast at Tiffany's,1961,A,115,Comedy,Drama,Romance,7.6,A young New York socialite becomes interested ...,76,Blake Edwards,Audrey Hepburn,George Peppard,Patricia Neal,Buddy Ebsen,166544,<NA>
996,Giant,1956,G,201,Drama,Western,None,7.6,Sprawling epic covering the life of a Texas ca...,84,George Stevens,Elizabeth Taylor,Rock Hudson,James Dean,Carroll Baker,34075,<NA>
997,From Here to Eternity,1953,Passed,118,Drama,Romance,War,7.6,"In Hawaii in 1941, a private is cruelly punish...",85,Fred Zinnemann,Burt Lancaster,Montgomery Clift,Deborah Kerr,Donna Reed,43374,30500000
998,Lifeboat,1944,None,97,Drama,War,None,7.6,Several survivors of a torpedoed merchant ship...,78,Alfred Hitchcock,Tallulah Bankhead,John Hodiak,Walter Slezak,William Bendix,26471,<NA>
